<a href="https://colab.research.google.com/github/sehee020512/20242R0136COSE47402/blob/master/final%20project%2012.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation

In [28]:
import kagglehub

path = kagglehub.dataset_download("kavyasreeb/hair-type-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/kavyasreeb/hair-type-dataset/versions/1


In [29]:
import shutil

# 원본 데이터 경로
source_path = "/root/.cache/kagglehub/datasets/kavyasreeb/hair-type-dataset/versions/1"

# 복사할 대상 경로
destination_path = "/content/hair-type-dataset"

# 데이터 복사
try:
    shutil.copytree(source_path, destination_path)
    print(f"데이터가 {destination_path}에 복사되었습니다.")
except Exception as e:
    print(f"복사 실패: {e}")

복사 실패: [Errno 17] File exists: '/content/hair-type-dataset'


In [30]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터 경로 설정
dataset_dir = "/content/hair-type-dataset/data"  # 클래스별 이미지가 있는 디렉토리
train_dir = "/content/train"  # train 데이터 저장 경로
test_dir = "/content/test"  # test 데이터 저장 경로

# train/test 디렉토리 생성
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# 이미지 확장자 필터
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

# 클래스별 train/test 데이터 분리
split_ratio = 0.8  # Train 80%, Test 20%
for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if not os.path.isdir(category_path):
        continue  # 디렉토리가 아니면 건너뜀

    # 클래스별 이미지 파일 가져오기
    images = [img for img in os.listdir(category_path) if img.lower().endswith(valid_extensions)]

    # 이미지가 없으면 건너뜀
    if len(images) == 0:
        print(f"'{category}' 폴더에 이미지가 없습니다. 건너뜁니다.")
        continue

    # train/test로 분리
    train_images, test_images = train_test_split(images, test_size=1 - split_ratio, random_state=42)

    # 클래스별 train/test 디렉토리 생성
    train_category_dir = os.path.join(train_dir, category)
    test_category_dir = os.path.join(test_dir, category)
    os.makedirs(train_category_dir, exist_ok=True)
    os.makedirs(test_category_dir, exist_ok=True)

    # train 이미지 이동
    for img in train_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(train_category_dir, img))

    # test 이미지 이동
    for img in test_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(test_category_dir, img))

    print(f"'{category}' 클래스에서 Train: {len(train_images)}개, Test: {len(test_images)}개 파일이 이동되었습니다.")

print("Train/Test 데이터 분리가 완료되었습니다!")

'dreadlocks' 클래스에서 Train: 354개, Test: 89개 파일이 이동되었습니다.
'Wavy' 클래스에서 Train: 263개, Test: 66개 파일이 이동되었습니다.
'Straight' 클래스에서 Train: 388개, Test: 97개 파일이 이동되었습니다.
'kinky' 클래스에서 Train: 173개, Test: 44개 파일이 이동되었습니다.
'curly' 클래스에서 Train: 411개, Test: 103개 파일이 이동되었습니다.
Train/Test 데이터 분리가 완료되었습니다!


In [31]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # DeiT 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 데이터 로드
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [32]:
!pip install pillow

In [33]:
from PIL import Image

## Load Model

In [34]:
!pip install transformers

In [35]:
import torch
import torch.nn as nn
import timm
import torchvision
import torchvision.transforms as T

In [36]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


In [37]:
model.head = nn.Linear(model.head.in_features, 5)

torch.nn.init.xavier_uniform_(model.head.weight)
torch.nn.init.zeros_(model.head.bias)

print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [38]:
model = model.to(device)  # GPU 또는 CPU로 모델 이동
img_tensor = img_tensor.to(device)  # GPU 또는 CPU로 텐서 이동

In [39]:
!pip install scikit-learn

In [40]:
# 필요한 라이브러리 임포트
from sklearn.metrics import accuracy_score  # 정확도 계산 함수

In [41]:
test_images = []
test_labels = []  # 실제 레이블 저장
predicted_labels = []  # 예측된 레이블 저장

# 하위 폴더를 순회하며 이미지와 레이블을 로드
try:
    label_mapping = {label: idx for idx, label in enumerate(sorted(os.listdir(test_dir)))}
    reverse_mapping = {v: k for k, v in label_mapping.items()}

    for category in os.listdir(test_dir):
        category_path = os.path.join(test_dir, category)
        if not os.path.isdir(category_path):
            continue

        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            if file_name.lower().endswith(valid_extensions):  # 유효한 이미지 파일인지 확인
                try:
                    # 폴더 이름을 레이블로 사용
                    label = label_mapping[category]
                    test_labels.append(label)

                    # 이미지 열기 및 전처리
                    img = Image.open(file_path).convert("RGB")
                    img_tensor = transform(img).unsqueeze(0).to(device)  # 배치 차원 추가

                    # 모델 예측
                    with torch.no_grad():
                        output = model(img_tensor)
                        predicted_class = torch.argmax(output, dim=1).item()

                    # 예측된 레이블 저장
                    predicted_labels.append(predicted_class)
                except Exception as e:
                    print(f"이미지 처리 실패: {file_name}, 오류: {e}")
except Exception as e:
    print(f"테스트 데이터 로드 중 오류 발생: {e}")

# 4. 정확도 계산 및 출력
if test_labels and predicted_labels:
    accuracy = accuracy_score(test_labels, predicted_labels)
    print(f"테스트 데이터 개수: {len(test_labels)}")
    print(f"정확도: {accuracy * 100:.2f}%")

    # 예측 결과 예시 출력
    for i in range(len(test_labels)):
        true_label = reverse_mapping[test_labels[i]]
        predicted_label = reverse_mapping[predicted_labels[i]]
        print(f"실제: {true_label} | 예측: {predicted_label}")
else:
    print("테스트 데이터가 비어 있거나 예측 결과가 없습니다.")

테스트 데이터 개수: 399
정확도: 23.31%
실제: dreadlocks | 예측: kinky
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Wavy
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: kinky
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: kinky
실제: dreadlocks | 예측: kinky
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: kinky
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: Straight
실제: dreadlocks | 예측: curly
실제: dreadlocks | 예측: Stra

## Fine-Tuning

In [42]:
import torch.optim as optim

In [43]:
# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

# 학습률 스케줄러 (옵션)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
num_epochs = 10  # 에포크 수

for epoch in range(num_epochs):
    # 모델 학습 모드
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파 및 손실 계산
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 역전파 및 옵티마이저 업데이트
        loss.backward()
        optimizer.step()

        # 손실과 정확도 업데이트
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%")

    # 학습률 스케줄러 업데이트 (옵션)
    scheduler.step()

# 학습 완료
print("Fine-Tuning 완료!")

Epoch [1/10], Loss: 0.5871, Accuracy: 77.14%
Epoch [2/10], Loss: 0.1078, Accuracy: 96.03%
Epoch [3/10], Loss: 0.0595, Accuracy: 98.05%
Epoch [4/10], Loss: 0.0226, Accuracy: 99.50%
Epoch [5/10], Loss: 0.0431, Accuracy: 98.55%
Epoch [6/10], Loss: 0.0184, Accuracy: 99.37%


In [ ]:
# 모델 평가 모드
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"테스트 데이터 정확도: {100.*correct/total:.2f}%")